##### 참고 url : https://progr-account.tistory.com/32
- 기업 개황 불러오기

In [1]:
### 필요한 모듈
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import xml.etree.ElementTree as ET

##### 기업 개황 불러오기 : 각각 검색하여 하나씩 조회하기

In [2]:
### 회사고유번호 데이터 불러오기
url = 'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key=3d218f72d3a558aa85ff91f2a0f2d720e3158223'
with urlopen(url) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('corp_num')

In [3]:
### 압축파일 안의 xml 파일 읽기
tree = ET.parse('./corp_num/CORPCODE.xml')
root = tree.getroot()

In [4]:
### 회사 이름으로 회사 고유번호 찾기
def find_corp_num(find_name):
    for country in root.iter("list"):
        if country.findtext("corp_name") == find_name:
            return country.findtext("corp_code")

In [5]:
find_corp_num('삼성전자')

'00126380'

In [6]:
### 모듈 import
import requests

def load_data(**kwargs):
    ### 본인의 인정키 입력
    crtfc_key = "3d218f72d3a558aa85ff91f2a0f2d720e3158223"
    ### load_data 함수 입력을 통해 corp_code 를 받기로 한다.
    corp_code = kwargs['corp_code']
    
    ###기업개황 요청하면
    if kwargs['request'] == 'company':
        
        ### 기업개황 요청 url
        url = 'https://opendart.fss.or.kr/api/company.json?crtfc_key='+crtfc_key+'&corp_code='+corp_code
        
        ### HTTP 요청
        r = requests.get(url)
        
        ### 요청한 데이터는 json형태이기 때문에 json으로 읽어줍니다.
        company_data = r.json()
        
        ### 기업개황을 요청했을 때 기업개황에 대한 자료를 반환합니다.
        return company_data

In [7]:
load_data(request='company', corp_code=find_corp_num('삼성전자'))

{'status': '000',
 'message': '정상',
 'corp_code': '00126380',
 'corp_name': '삼성전자(주)',
 'corp_name_eng': 'SAMSUNG ELECTRONICS CO,.LTD',
 'stock_name': '삼성전자',
 'stock_code': '005930',
 'ceo_nm': '한종희, 경계현',
 'corp_cls': 'Y',
 'jurir_no': '1301110006246',
 'bizr_no': '1248100998',
 'adres': '경기도 수원시 영통구  삼성로 129 (매탄동)',
 'hm_url': 'www.samsung.com/sec',
 'ir_url': '',
 'phn_no': '02-2255-0114',
 'fax_no': '031-200-7538',
 'induty_code': '264',
 'est_dt': '19690113',
 'acc_mt': '12'}

##### 기업개황 불러오기 : 상장된 모든 기업의 개황정보 다운로드 -> pickle을 이용하여 파일 내에서 검색

In [8]:
### stock_code 가 있는 기업의 corp_code만 code_list에 추가
### 상장회사가 아닌경우 기업개황 외의 API 서비스가 제공되지 않기때문
code_list=[]
for x in range(len(root)):
    
    #root[0][0].tag = 'corp_code'
    #root[0][1].tag = 'corp_name'
    #root[0][2].tag = 'stock_code'
    
    ### stock_cdoe 가 없는게 아니라면 => stock_code가 있다면
    if root[x][2].text != " ":
        code_list.append(root[x][0].text)
### stock_code 가 있는 기업의 갯수 확인        
len(code_list)

3588

In [9]:
### xml 모듈 import
import xml.etree.ElementTree as ET

import warnings
warnings.filterwarnings('ignore')

### xml 파일 읽기
tree = ET.parse('./corp_num/CORPCODE.xml')
root = tree.getroot()

In [10]:
### stock_code 가 있는 기업의 corp_code만 code_list에 추가
### 상장회사가 아닌경우 기업개황 외의 API 서비스가 제공되지 않기때문
code_list=[]
for x in range(len(root)):
    
    #root[0][0].tag = 'corp_code'
    #root[0][1].tag = 'corp_name'
    #root[0][2].tag = 'stock_code'
    
    ### stock_cdoe 가 없는게 아니라면 => stock_code가 있다면
    if root[x][2].text is not " ":
        code_list.append(root[x][0].text)
### stock_code 가 있는 기업의 갯수 확인        
len(code_list)

3588

In [11]:
### HTTP 요청
import requests

### 기업개황 불러오기 함수
def load_data(corp_code):
    ### 본인의 인정키 입력
    crtfc_key = "3d218f72d3a558aa85ff91f2a0f2d720e3158223"
    ### 기업개황 요청 url
    url = 'https://opendart.fss.or.kr/api/company.json?crtfc_key='+crtfc_key+'&corp_code='+corp_code
    ### HTTP 요청
    r = requests.get(url, verify=False)
    ### 요청한 데이터는 json형태이기 때문에 json으로 읽어줍니다.
    company_data = r.json()
    ### 기업개황을 요청했을 때 기업개황에 대한 자료를 dict type 으로 반환합니다.
    return company_data

In [12]:
### 진행상황을 표시해줄 모듈 import
from tqdm import tqdm_notebook

### 기업 개황을 담을 list 생성
company_condition_list = []

### 3179개 기업황을 불러오는데는 시간이 오래 걸리므로 tqdm 모듈을 사용해 진행상황을 표시하겠습니다.
for corp_code in tqdm_notebook(code_list):
    ### load_data 함수를 통해 불러온 기업개황 dict를 company_dict을 변수로 저장합니다.
    company_dict = load_data(corp_code)
    ### 기업개황 리스트에 company_dict를 추가합니다.
    company_condition_list.append(company_dict)

  0%|          | 0/3588 [00:00<?, ?it/s]

In [13]:
### pickle 모듈 import
import pickle

### pickle 모듈을 통해 list 저장
with open('./corp_num/company_condition_list.txt','wb') as f:
       pickle.dump(company_condition_list,f)

In [14]:
### Pnadas DataFrame 모듈 import
import pandas as pd

pd_data = pd.DataFrame.from_dict(company_condition_list)
pd_data

,status,message,corp_code,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,ir_url,phn_no,fax_no,induty_code,est_dt,acc_mt
0,000,정상,00260985,한빛네트,"HanbitNet, Inc.",한빛네트,036720,강승환,E,1101111443079,2088124283,서울특별시 종로구 익선동 30-6,www.hanbitnet.com,,02-3668-0114,02-3668-0112,58221,19970722,12
1,000,정상,00264529,(주)엔플렉스,"Nplex,Inc.",엔플렉스,040130,이 정 학,E,1101111601750,2208156354,서울특별시 강남구 논현동 98 동익빌딩 2층,www.gamekorea.net,,(02) 3453-7111,(02) 3453-7345,58221,19981030,12
2,000,정상,00358545,(주)동서정보기술,"Dong Seo Information Technology Co., Ltd.",동서정보기술,055000,이희봉,E,1101111259525,2208103653,서울특별시 강남구 대치동 997-4,http://www.dsit.co.kr,,02-565-4510 (220),02-3453-4552,58221,19960410,12
3,000,정상,00231567,주식회사 애드모바일,,애드모바일,032600,김용석,E,1101111166837,1208151156,서울특별시 강서구 가양동 1458-9 동남빌딩 2층,www.kdline.co.kr,,02-518-0101 (변광섭),02-518-0133,58221,19950614,12
4,000,정상,00247939,(주)씨모스,"creative master of semiconductor. co.,ltd",씨모스,037600,유인엽,E,1201110118077,1398120545,인천광역시 남동구 고잔동 685-6,www.cmos21.co.kr,,032-814-0981-4,032-814-0985,29294,19950509,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3583,000,정상,00296078,(주)에이피에스,APS Inc.,APS,054620,정기로,K,1101111315286,2298122962,경기도 화성시 동탄면 동탄산단9길 23-12 (동탄면),www.apsholdings.co.kr,,031-776-1800,031-776-1900,64992,19960829,12
3584,000,정상,01118643,(주)로보쓰리에이아이앤로보틱스,"Robo3 Ai & Robotics co., Ltd",로보쓰리에이아이앤로보틱스,238500,"심종헌, 박영근(각자 대표이사)",N,1101112858483,2068184723,서울특별시 강남구 봉은사로18길 47(역삼동622-7) 전층,www.robo3.com,www.robo3.com,02-544-9145,02-544-9146,739,20030917,12
3585,000,정상,00624518,(주)다산솔루에타,"Dasan Solueta Co.,Ltd.",다산솔루에타,154040,양삼주,K,1351110065750,1238189952,경기도 화성시 마도면 해운로 498-42 -,www.solueta.com,http://www.solueta.com/kr/m51.php,031-508-2655,031-508-2606,2629,20030627,12
3586,000,정상,01065679,(주)알비케이그룹,RBK GROUP INC.,알비케이그룹,215790,한혁,K,1101115658640,4478700025,인천광역시 연수구 송도미래로 30 송도스마트밸리 E-2206,,,02-750-5644,,264,20150306,12


In [15]:
with open('./corp_num/company_condition_list.txt','rb') as f:
	company_condition_list=pickle.load(f) ### 상장기업 기업개황을 company_condition_list에 저장
len(company_condition_list)

3588

In [16]:
print(type(company_condition_list[0])) ### list 요소의 type 확인
print(company_condition_list[0].keys()) ### dict key list 확인

<class 'dict'>
dict_keys(['status', 'message', 'corp_code', 'corp_name', 'corp_name_eng', 'stock_name', 'stock_code', 'ceo_nm', 'corp_cls', 'jurir_no', 'bizr_no', 'adres', 'hm_url', 'ir_url', 'phn_no', 'fax_no', 'induty_code', 'est_dt', 'acc_mt'])


In [17]:
### 함수 생성 : 조건에 맞는 회사의 기업개황 list 반환
def find_condition(**kwargs):
    
    ### 저장된 상장기업 개황 list 불러오기
    with open('company_condition_list.txt','rb') as f:
        company_condition_list=pickle.load(f)
    
    ### 결과값을 반환할 list
    return_list = []
    
    try:
        ### 필수입력사항
        key = kwargs['key'] # 기업개황 dict 의 조건에 맞는 key
        value = kwargs['value'] # 기업개황 dict 의 조건에 맞는 value
    except:
        key = None
        value  = None
    
    ### 선택입력사항
    ### select == same 이면 value 와 일치하는것만
    ### select is not same 이면 value 가 포함된 모든것을 return 합니다.
    try:
        select = kwargs['select']
        print(select)
    except:
        select = None

    ### select == all 이면 모든 기업개황 반환
    if select == 'all': 
        return_list = company_condition_list

    ### select == same 이면 value 와 일치하는것만
    elif select == 'same':
        for company_condition in  company_condition_list:
            if value == company_condition[key]:
                return_list.append(company_condition)

    ### select is not same 이면 value 가 포함된 모든것
    else :
        for company_condition in company_condition_list:
            if value in company_condition[key]:
                return_list.append(company_condition)

    return return_list

In [18]:
# 모든 회사의 기업개황 불러오기
find_condition(select='all')

FileNotFoundError: [Errno 2] No such file or directory: 'company_condition_list.txt'

In [ ]:
# 주소를 포함한 회사 검색
result_list = find_condition(key = 'adres', value='부산진구')
print(len(result_list))
result_list

2


[{'status': '000',
  'message': '정상',
  'corp_code': '00100939',
  'corp_name': '강남제비스코(주)',
  'corp_name_eng': 'KANGNAM JEVISCO CO.,LTD',
  'stock_name': '강남제비스코',
  'stock_code': '000860',
  'ceo_nm': '황익준,김재현(각자대표이사)',
  'corp_cls': 'Y',
  'jurir_no': '1801110003391',
  'bizr_no': '6058103330',
  'adres': '부산광역시 부산진구 중앙대로 639 18층(범천동, 엠디엠타워)',
  'hm_url': 'www.jevisco.com',
  'ir_url': '',
  'phn_no': '051-892-4221',
  'fax_no': '051-892-5576',
  'induty_code': '20411',
  'est_dt': '19520716',
  'acc_mt': '12'},
 {'status': '000',
  'message': '정상',
  'corp_code': '00646732',
  'corp_name': '(주)진양홀딩스',
  'corp_name_eng': 'CHINYANG HOLDINGS CORPORATION',
  'stock_name': '진양홀딩스',
  'stock_code': '100250',
  'ceo_nm': '양준영, 최창호(각자대표이사)',
  'corp_cls': 'Y',
  'jurir_no': '1801110618091',
  'bizr_no': '6058186253',
  'adres': '부산광역시 부산진구 새싹로 73 -',
  'hm_url': 'cyholdings.kr',
  'ir_url': '',
  'phn_no': '051-809-8813',
  'fax_no': '0553822592',
  'induty_code': '222',
  'est_dt': '20080

In [ ]:
# 업종코드가 일치하는 회사 검색
result_list = find_condition(key = 'induty_code', value='30', select = 'same')
print(len(result_list))
result_list

same
7


[{'status': '000',
  'message': '정상',
  'corp_code': '00679730',
  'corp_name': '(주)파브코',
  'corp_name_eng': 'PYUNG HWA ANTI-VIBRATION SYSTEM CO., LTD.',
  'stock_name': '파브코',
  'stock_code': '101990',
  'ceo_nm': '조남일',
  'corp_cls': 'E',
  'jurir_no': '1701110355257',
  'bizr_no': '5148164841',
  'adres': '대구광역시 달성군 논공읍 논공로 597',
  'hm_url': 'pavco.co.kr',
  'ir_url': '',
  'phn_no': '053-610-7000',
  'fax_no': '053-615-0566',
  'induty_code': '30',
  'est_dt': '20080508',
  'acc_mt': '12'},
 {'status': '000',
  'message': '정상',
  'corp_code': '00106650',
  'corp_name': '(주)카스코',
  'corp_name_eng': 'KOREA AUTOMOTIVE SYSTEMS CO.,LTD',
  'stock_name': '카스코',
  'stock_code': '005330',
  'ceo_nm': '이정',
  'corp_cls': 'E',
  'jurir_no': '1942110002593',
  'bizr_no': '6098106829',
  'adres': '경상남도 창원시 성산동 72번지',
  'hm_url': 'www.kasco21.co.kr',
  'ir_url': '',
  'phn_no': '055-268-3500',
  'fax_no': '0551-261-3340',
  'induty_code': '30',
  'est_dt': '19671209',
  'acc_mt': '12'},
 {'stat

In [ ]:
# 유가증권 시장에 속하는 회사의 기업개황
result_list = find_condition(key = 'corp_cls', value='Y')
len(result_list)
result_list

[{'status': '000',
  'message': '정상',
  'corp_code': '00600013',
  'corp_name': '미래에셋맵스 아시아퍼시픽 부동산공모 1호 투자회사',
  'corp_name_eng': 'MIRAEASSETMAPS ASIA PACIFIC REAL ESTATE 1 INVESTMENT COMPANY',
  'stock_name': '맵스리얼티1',
  'stock_code': '094800',
  'ceo_nm': '미래에셋자산운용(주)',
  'corp_cls': 'Y',
  'jurir_no': '1101113595852',
  'bizr_no': '1078686211',
  'adres': '서울 영등포구 여의도동 미래에셋빌딩 45-1',
  'hm_url': 'www.miraeasset.com',
  'ir_url': '',
  'phn_no': '02-1577-1640',
  'fax_no': '02-769-7816',
  'induty_code': '642',
  'est_dt': '20070109',
  'acc_mt': '12'},
 {'status': '000',
  'message': '정상',
  'corp_code': '00435297',
  'corp_name': '맥쿼리한국인프라투융자회사',
  'corp_name_eng': 'Macquarie Korea Infrastructure Fund',
  'stock_name': '맥쿼리인프라',
  'stock_code': '088980',
  'ceo_nm': '맥쿼리자산운용(주)',
  'corp_cls': 'Y',
  'jurir_no': '1101112672586',
  'bizr_no': '1048177509',
  'adres': '서울특별시 종로구 우정국로 26 A동 18층(공평동, 센트로폴리스)',
  'hm_url': 'www.macquarie.com/kr/kr/mkif/index.html',
  'ir_url': '',
  'phn